In [1]:
from new_model.model_utils import resnet_shorten, hybrid_LSTM_training, _classifier, convolutional_block
from keras.layers import LSTM, Reshape, Input
from keras.models import Model
import keras
from training_utils import save_model, DataGenerator, generate_dataset

import time
import pickle

Using TensorFlow backend.


# Global constant

In [2]:
IMG_SHAPE = (200, 200, 1)
LSTM_DIM_HIDDEN = 64*2
LEN_SPATIAL_HISTORY = 4
NUM_CLASS = 73

# Name of previously trained weights

In [3]:
shared_encoder_file = "shared_encoder_resnet8_2019_05_21_17_28.h5"
shared_lstm_file = "lstm_weights_2019_05_21_17_28.p"
shared_classifier_file = ""

___

# 2.Model Definition

**------------------------**
## 2.1 Encoder

### 2.1.1 Shared encoder

In [4]:
shared_encoder = resnet_shorten(IMG_SHAPE, model_name="shared_encoder")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
shared_encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 100, 100, 32) 832         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 49, 49, 32)   0           conv_0[0][0]                     
__________________________________________________________________________________________________
bn_1_a (BatchNormalization)     (None, 49, 49, 32)   128         max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
activation

### 2.1.2 Separate encoder

This is a convolution block of ResNet.

In [6]:
def _separate_encoder(input_shape, num_filters, shape_filters, strides, stage, model_name):
    """
    Create a model from the function named "convolutional_block". This model is later used as a layer
    in the full hybrid model
    
    Input: 
        input_shape (tuple): shape of feature vectors created by shared_encoder
        num_filters (list): number of filters of each Conv2D layer of this model
        strides (list): size of strides of each Conv2D layer
        stage: must set to be None
        model_name (str):
        
    Output:
        keras Model instance
    """
    assert(stage is None)
    
    # define input
    X_input = Input(shape=input_shape)
    
    # pass input through a convolutional block of ResNet
    X = convolutional_block(X_input, num_filters, shape_filters, strides, stage=None, model_name=model_name)
    
    # define model
    model = Model(inputs=[X_input], outputs=[X], name=model_name)
    
    return model
    

In [7]:
# define config of separate encoder
s_input_shape = (13, 13, 64)  # shape of output of shared encoder
s_num_filters = [128, 128, 128]
s_shape_filters = [3, 3, 1]
s_strides = [2, 1, 2]

sep_encoder_list = [_separate_encoder(s_input_shape, s_num_filters, s_shape_filters, s_strides, 
                                      stage=None, model_name="sep_en_%d" % i) 
                   for i in range(LEN_SPATIAL_HISTORY)]

In [8]:
sep_encoder_list[0].summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 13, 13, 64)   0                                            
__________________________________________________________________________________________________
sep_en_0_bn_a (BatchNormalizati (None, 13, 13, 64)   256         input_2[0][0]                    
__________________________________________________________________________________________________
activation_5 (Activation)       (None, 13, 13, 64)   0           sep_en_0_bn_a[0][0]              
__________________________________________________________________________________________________
sep_en_0_conv_a (Conv2D)        (None, 7, 7, 128)    73856       activation_5[0][0]               
__________________________________________________________________________________________________
sep_en_0_b

**------------------------**
## 2.2 Decoder

### 2.2.1 Define LSTM

In [9]:
LSTM_cell = LSTM(LSTM_DIM_HIDDEN, return_state=True)

### 2.2.2 Define Classifier

In [10]:
classifier = _classifier(input_shape=(LSTM_DIM_HIDDEN, ), num_class=NUM_CLASS)

In [11]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 73)                18761     
_________________________________________________________________
activation_13 (Activation)   (None, 73)                0         
Total params: 51,785
Trainable params: 51,785
Non-trainable params: 0
_________________________________________________________________


In [12]:
reshapor = Reshape((1, -1))

## 2.2 Full Model

In [13]:
flattener = keras.layers.Flatten()
activator = keras.layers.Activation('relu')


def model_shared_private_encoder(image_shape, shared_encoder, sep_encoder_list, LSTM_cell, 
                                 LSTM_dim_hidden_state, Ty):
    """
    Define full model with both shared & private encoder
    
    Input:
        image_shape (tuple): shape of input image
        shared_encoder (keras.Model): shared model used to extract low level feature vector from input image
        sep_encoder_list (list): list of keras.Model storing separate encoder
        LSTM_cell (keras.layers): shared LSTM layer
        LSTM_dim_hidden_state (int): dimension of LSTM_cell's hidden state
        Ty (int): length of spatial history
    
    Output:
        keras Model instance
    """
    # Input layer
    X_input_list = [Input(shape=image_shape) for i in range(Ty)]
    
    # pass each input through shared encoder
    shared_encoded_X = [shared_encoder(X) for X in X_input_list]
    
    # pass each encoded_X through its own convolution block
    separate_encoded_X = [separate_encoder(X) 
                          for separate_encoder, X in zip(sep_encoder_list, shared_encoded_X)]
    
    # initialize input & cell state
    a_0 = Input(shape=(LSTM_dim_hidden_state, ))  
    c_0 = Input(shape=(LSTM_dim_hidden_state, ))
    
    a = a_0
    c = c_0
    
    outputs = []
    
    # Decode
    for encoded_X in separate_encoded_X:
        # flatten & activate encoded_X 
        X = flattener(encoded_X)
        X = activator(X)
        
        # perform 1 step of LSTM cell
        X = reshapor(encoded_X)
        a, _, c = LSTM_cell(X, initial_state=[a, c])
        
        # apply regressor to the hidden state of LSTM_cell
        out = classifier(a)
        
        # append out to outputs
        outputs.append(out)
    
    # define model
    model = Model(inputs=X_input_list + [a_0, c_0], outputs=outputs)
    
    return model


In [14]:
hybrid_model = model_shared_private_encoder(IMG_SHAPE, shared_encoder, sep_encoder_list, LSTM_cell, 
                                            LSTM_DIM_HIDDEN, LEN_SPATIAL_HISTORY)

In [15]:
hybrid_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
shared_encoder (Model)          (None, 13, 13, 64)   78560       input_7[0][0]                    
                                                                 input_8[0][0]                    
                                                                 input_9[0][0]                    
                                                                 input_10[0][0]                   
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 200, 200, 1)  0                                            
__________

## 2.3 Load weights & compile model

In [16]:
# Load shared_encoder
shared_encoder.load_weights("./new_model/weights/shared_encoder/%s" % shared_encoder_file, by_name=True)

# Load shared LSTM
with open('./new_model/weights/shared_lstm/%s' % shared_lstm_file, 'rb') as fp:
    lstm_weights_dict = pickle.load(fp)
    
lstm_weights = []
for k in lstm_weights_dict.keys():
    lstm_weights.append(lstm_weights_dict[k])
    
LSTM_cell.set_weights(lstm_weights)

# Load shared classifier
classifier.load_weights("./new_model/weights/shared_classifier/%s" % shared_classifier_file)

In [17]:
otim = keras.optimizers.Adam(lr=0.25, decay=0.001)
hybrid_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

---

# 3. Training

In [18]:
batch_size = 32

param_train = {'img_shape': IMG_SHAPE, 
             'Ty': LEN_SPATIAL_HISTORY, 
             'num_class': NUM_CLASS, 
             'batch_size': batch_size, 
             'shuffle': True, 
             'additional_input_for_LSTM': True, 
             'LSTM_dim_hidden_states': LSTM_DIM_HIDDEN}

train_gen = DataGenerator("./new_data/widthen_bin_training_CH2_only.csv", **param_train)

param_val = {'img_shape': IMG_SHAPE, 
             'num_class': NUM_CLASS, 
             'Ty': LEN_SPATIAL_HISTORY, 
             'LSTM_dim_hidden_states': LSTM_DIM_HIDDEN, 
             'color_img': False}
X_val, y_val = generate_dataset("./new_data/widthen_bin_validation_CH2_only.csv", **param_val)

In [26]:
time_str = time.strftime("%Y_%m_%d_%H_%M")
tb_callback = keras.callbacks.TensorBoard(log_dir='./logs/' + time_str,  
                                          batch_size=batch_size, 
                                          update_freq='epoch')


In [27]:
hybrid_model.fit_generator(train_gen,
                           epochs=20,
                           validation_data=(X_val, y_val),
                           initial_epoch=10,
                           callbacks=[tb_callback])

Epoch 11/20
569/569 [==============================] - 204s 359ms/step - loss: 9.1436 - model_1_loss: 2.2973 - model_1_acc: 0.3063 - model_1_acc_1: 0.2912 - model_1_acc_2: 0.3133 - model_1_acc_3: 0.3011 - val_loss: 9.1558 - val_model_1_loss: 2.2667 - val_model_1_acc: 0.3121 - val_model_1_acc_1: 0.2790 - val_model_1_acc_2: 0.3284 - val_model_1_acc_3: 0.3037
Epoch 12/20
569/569 [==============================] - 201s 354ms/step - loss: 9.1229 - model_1_loss: 2.2828 - model_1_acc: 0.3008 - model_1_acc_1: 0.2942 - model_1_acc_2: 0.2976 - model_1_acc_3: 0.2901 - val_loss: 9.1008 - val_model_1_loss: 2.2769 - val_model_1_acc: 0.3185 - val_model_1_acc_1: 0.3165 - val_model_1_acc_2: 0.3289 - val_model_1_acc_3: 0.2869
Epoch 13/20
569/569 [==============================] - 202s 355ms/step - loss: 9.1394 - model_1_loss: 2.2868 - model_1_acc: 0.3029 - model_1_acc_1: 0.2905 - model_1_acc_2: 0.2965 - model_1_acc_3: 0.3094 - val_loss: 9.1101 - val_model_1_loss: 2.2544 - val_model_1_acc: 0.3151 - val_m

# 4. Save weights

In [28]:
# save shared_encoder 
shared_encoder.save_weights("./new_model/weights/shared_encoder/shared_encoder_resnet8_%s.h5" % time_str)

# save shared_lstm
lstm_w = LSTM_cell.get_weights()
lstm_w_dict ={}
lstm_w_dict['0'] = lstm_w[0]
lstm_w_dict['1'] = lstm_w[1]
lstm_w_dict['2'] = lstm_w[2]

with open("./new_model/weights/shared_lstm/shared_lstm_weights_%s.p" % time_str, 'wb') as fp:
    pickle.dump(lstm_w_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

# save separate encoder
for i, sep_encoder in enumerate(sep_encoder_list):
    sep_encoder.save_weights("./new_model/weights/separate_encoder/sep_encoder_%d_%s.h5" % (i, time_str))

# save classifier
classifier.save_weights("./new_model/weights/shared_classifier/classifier_%s.h5" % time_str)